In [15]:
import pandas as pd
import qiime2 as q2
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
data_dir = 'data'
source = 'data/raw'
! mkdir -p data data/raw


Upload the files from Rodrigo to the raw directory

## Extract compressed files from .tar.gz archives

In [60]:
import os
import pandas as pd

data_dir = "data"
source = "data/raw"

# load metadata
metadata_df = pd.read_csv(f"{source}/merged_metadata_filtered.tsv", sep="\t", index_col=0)

records = []

for technique in os.listdir(source):
    tech_path = os.path.join(source, technique)
    if not os.path.isdir(tech_path):
        continue
    for sample_id in os.listdir(tech_path):
        sample_path = os.path.join(tech_path, sample_id)
        if not os.path.isdir(sample_path):
            continue
        for f in os.listdir(sample_path):
            if f.endswith((".fa", ".fasta")):
                abs_path = os.path.abspath(os.path.join(sample_path, f))
                if sample_id in metadata_df.index:
                    base = os.path.splitext(f)[0]  # remove .fa or .fasta
                    new_id = f"{sample_id}_{base}"
                    records.append((new_id, abs_path))

# build dataframe
manifest_df = pd.DataFrame.from_records(records, columns=["sample-id", "absolute-filepath"])
manifest_df.set_index("sample-id", inplace=True)

print(manifest_df.head())

# export MANIFEST
manifest_df.to_csv(f"{data_dir}/MANIFEST", sep="\t")


                                                  absolute-filepath
sample-id                                                          
M004_metabat2.11  /home/jovyan/Interplanetary_Microbiome/data/ra...
M004_metabat2.13  /home/jovyan/Interplanetary_Microbiome/data/ra...
M004_metabat2.28  /home/jovyan/Interplanetary_Microbiome/data/ra...
M004_metabat2.10  /home/jovyan/Interplanetary_Microbiome/data/ra...
M004_metabat2.4   /home/jovyan/Interplanetary_Microbiome/data/ra...


In [42]:
######create Merged_df for exploration######
metadata_df['sample_id'] = metadata_df.index
merged_df=pd.merge(metadata_df, manifest_df, how = 'right', left_on = 'sample_ID', right_on = 'sample-id')
merged_df.to_csv(f"{data_dir}/metadata_merged.csv", index=False)

## EXPLORATION

In [49]:
merged_df.columns

Index(['samp_country', 'category', 'fermented_food_type', 'sample_id',
       'absolute-filepath'],
      dtype='object')

In [51]:
merged_df['samp_country'].value_counts()

samp_country
Thailand       621
Laos           568
Benin          200
Slovenia        19
Germany         10
Switzerland      8
Name: count, dtype: int64

In [54]:
merged_df['category'].value_counts()

category
fermented fish          772
fermented meat          363
fermented legumes       127
fermented roots         120
fermented vegetables     33
fermented cereal         11
Name: count, dtype: int64

In [ ]:
! qiime tools import \
    --type "SampleData[SequencesWithQuality]" \
    --input-format SingleEndFastqManifestPhred33V2 \
    --input-path $data_dir/MANIFEST \
    --output-path $data_dir/demux_seqs.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
